In [46]:
import pandas as pd

In [183]:
import re

In [318]:
import copy

In [240]:
df_his_weight=pd.read_csv("../../raw/stock/sp500_historical_weight.csv")

In [48]:
df_com_CIK_SIC_NAICS_rele=pd.read_csv("../../processed/com_CIK_SIC_NAICS_rele.csv")

In [241]:
df_his_weight

,date,rank,ticker,name,sector,weight
0,2009-01-30,1.0,XOM,EXXON MOBIL CORP,Energy,5.403875
1,2009-01-30,2.0,PG,PROCTER & GAMBLE,Consumer Staples,2.260313
2,2009-01-30,3.0,JNJ,JOHNSON & JOHNSON,Health Care,2.223419
3,2009-01-30,4.0,T,AT&T INC,Telecommunication Services,2.015346
4,2009-01-30,5.0,CVX,CHEVRON CORP,Energy,1.990291
...,...,...,...,...,...,...
1527463,2024-10-30,499.0,BEN,FRANKLIN RESOURCES INC,Financials,0.013080
1527464,2024-10-30,500.0,PARA,PARAMOUNT GLOBAL CLASS B,Communication,0.012860
1527465,2024-10-30,501.0,FOX,FOX CORP CLASS B,Communication,0.010430
1527466,2024-10-30,502.0,AMTM,AMENTUM HOLDINGS INC,Industrials,0.007630


In [50]:
df_com_CIK_SIC_NAICS_rele

,cik,name,entityType,sic,sicDescription,ownerOrg,insiderTransactionForOwnerExists,insiderTransactionForIssuerExists,tickers,exchanges,...,mailing_stateOrCountry,mailing_zipCode,mailing_stateOrCountryDescription,business_street1,business_street2,business_city,business_stateOrCountry,business_zipCode,business_stateOrCountryDescription,naics
0,320193,Apple Inc.,operating,3571.0,Electronic Computers,06 Technology,0,1,AAPL,Nasdaq,...,CA,95014,CA,ONE APPLE PARK WAY,NaN,CUPERTINO,CA,95014,CA,334111
1,789019,MICROSOFT CORP,operating,7372.0,Services-Prepackaged Software,06 Technology,1,1,MSFT,Nasdaq,...,WA,98052-6399,WA,ONE MICROSOFT WAY,NaN,REDMOND,WA,98052-6399,WA,"511210,334614"
2,1045810,NVIDIA CORP,operating,3674.0,Semiconductors & Related Devices,04 Manufacturing,1,1,NVDA,Nasdaq,...,CA,95051,CA,2788 SAN TOMAS EXPRESSWAY,NaN,SANTA CLARA,CA,95051,CA,334413
3,1652044,Alphabet Inc.,operating,7370.0,"Services-Computer Programming, Data Processing...",06 Technology,1,1,"GOOGL,GOOG","Nasdaq,Nasdaq",...,CA,94043,CA,1600 AMPHITHEATRE PARKWAY,NaN,MOUNTAIN VIEW,CA,94043,CA,"518210,541511"
4,1326801,"Meta Platforms, Inc.",operating,7370.0,"Services-Computer Programming, Data Processing...",06 Technology,0,1,META,Nasdaq,...,CA,94025,CA,1 META WAY,NaN,MENLO PARK,CA,94025,CA,"518210,541511"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1535,1866226,Welsbach Technology Metals Acquisition Corp.,operating,3690.0,"Miscellaneous Electrical Machinery, Equipment ...",04 Manufacturing,0,1,"WTMA,WTMAR,WTMAU","Nasdaq,Nasdaq,Nasdaq",...,IL,60640,IL,4422 N. RAVENSWOOD AVE #1025,NaN,CHICAGO,IL,60640,IL,"333618,333318,333992,335129,335999"
1536,1987240,SCHMID Group N.V.,other,3590.0,Misc Industrial & Commercial Machinery & Equip...,06 Technology,0,0,"SHMD,SHMDW","Nasdaq,Nasdaq",...,2M,72250,Germany,ROBERT-BOSCH-STR. 32-36,NaN,FREUDENSTADT,2M,72250,Germany,"332710,336390,333318,332999,332813,333999"
1537,1940674,SMX (Security Matters) Public Ltd Co,other,3590.0,Misc Industrial & Commercial Machinery & Equip...,06 Technology,0,0,"SMX,SMXWW","Nasdaq,Nasdaq",...,L2,D04 T4A6,Ireland,"MESPIL BUSINESS CENTRE, MESPIL HOUSE",SUSSEX ROAD,DUBLIN 4,L2,D04 T4A6,Ireland,"332710,336390,333318,332999,332813,333999"
1538,1995194,Heramba Electric plc,other,3690.0,"Miscellaneous Electrical Machinery, Equipment ...",04 Manufacturing,0,0,"PITA,PITAW","Nasdaq,Nasdaq",...,L2,D02 R296,Ireland,70 SIR JOHN ROGERSON'S QUAY,NaN,DUBLIN 2,L2,D02 R296,Ireland,"333618,333318,333992,335129,335999"


In [242]:
lst_weight_ticker=list(df_his_weight.ticker.unique())
lst_weight_ticker

['XOM',
 'PG',
 'JNJ',
 'T',
 'CVX',
 'MSFT',
 'GEC',
 'IBM',
 'WMT',
 'PFE',
 'JPM',
 'CSCO',
 'ABT',
 'HPQ',
 'KO',
 'VZ',
 'GOOGL',
 'AAPL*',
 'WFC*',
 'PEP',
 'PM',
 'INCO',
 'COP',
 'ORCL',
 'MCD',
 'MRK',
 'AMGN',
 'WYE',
 'QCI',
 'SLBA',
 'GILD',
 'UTX',
 'OXY',
 'BMY',
 'UPS',
 'CMCSA',
 'BAC',
 'MON',
 'KFT',
 'CVS',
 'DIS',
 'MDT',
 'MMM',
 'LLY',
 'HD',
 'BAX',
 'GS',
 'EXC',
 'UNH',
 'MO',
 'TWX',
 'CL',
 'BA',
 'BK',
 'LOM',
 'DVN',
 'WAG',
 'LOW',
 'USB',
 'SO',
 'APA',
 'EMR',
 'CELG',
 'ALD',
 'TGT',
 'MWZ',
 'PA9',
 'EMC',
 'MHS',
 'UNP',
 'GD',
 'MS',
 'XTO',
 'KMB',
 'ANTM',
 'NEE',
 'RTN1',
 'DD',
 'D',
 'GIS',
 'COST',
 'AEC1',
 'TXN',
 '0R01',
 'COV',
 'NEM',
 'MRO',
 'PX',
 'DUK',
 'AMZN',
 'GENZ',
 'CAT',
 'BNI',
 'ADP',
 'NKE',
 'BDX',
 'ADM*',
 'EOG',
 'AAZ',
 'DELL',
 'AABA',
 'PEG',
 'FDX',
 'HES',
 'NTH',
 'GLW',
 'HAL',
 'WM',
 'FE',
 'CB',
 'TMO',
 'DE',
 'NCRA',
 'KR',
 'ETR',
 'DHR',
 'AET',
 'NSC',
 'BIIB',
 'PCG',
 'PNC',
 'CAH',
 'SYY',
 'BSX',
 'ESR

In [243]:
df_his_weight[df_his_weight.name=="AFLAC INC"]

,date,rank,ticker,name,sector,weight
154,2009-01-30,155.0,8686,AFLAC INC,Financials,0.150114
684,2009-02-27,185.0,8686,AFLAC INC,Financials,0.121656
1176,2009-03-31,177.0,8686,AFLAC INC,Financials,0.130317
1634,2009-04-30,135.0,8686,AFLAC INC,Financials,0.177942
2114,2009-05-29,115.0,8686,AFLAC INC,Financials,0.206024
...,...,...,...,...,...,...
1525122,2024-10-24,170.0,AFL,AFLAC INC,Financials,0.114650
1525628,2024-10-25,173.0,AFL,AFLAC INC,Financials,0.112370
1526131,2024-10-28,173.0,AFL,AFLAC INC,Financials,0.112110
1526633,2024-10-29,172.0,AFL,AFLAC INC,Financials,0.112150


In above, we see a list of strings each representing a ticker of a company. But some tickers have "*", ".", and space in it, we need to beware of that. Furthermore, there are companies with more than one tickers, we need to organize them to make sure they do not get lost. Ticker can be weird, look here https://en.wikipedia.org/wiki/Ticker_symbol. Let's try to list all the tickers with weird symbol (other than numbers and letters) first. 

In [244]:
reg_allow=re.compile("^[A-Za-z0-9]*$")
lst_weight_ticker_weird=[tic for tic in lst_weight_ticker if not bool(reg_allow.fullmatch(tic))]

In [245]:
lst_weight_ticker_weird

['AAPL*',
 'WFC*',
 'ADM*',
 'PRU*',
 'FCX*',
 'AIG*',
 'MIL.',
 'NSM.2',
 'GAS.2',
 'TWX*',
 'MOS*',
 'KMI WS',
 'S*',
 'FB*',
 'GOOG*',
 'BF/B',
 'BF.B',
 'TLN WI',
 'GCI.WI',
 'BLD WI',
 'CC WI',
 'CCP W',
 'SYF-W',
 'UAC/C',
 '-',
 'HWM WI',
 'ARNC WI',
 'RTX US',
 'OXY WS WI',
 'OXY WS',
 'VTRS US',
 'MBC WI']

According to wiki, there is no standard meaning for "*" at the end, and " ", "." alike only serve as separaters, so we can extract the company ticker by extracting till the first weird symbol. BTW, what is a "-" only ticker? 

In [246]:
df_his_weight[df_his_weight.ticker=="-"]

,date,rank,ticker,name,sector,weight
543629,2016-07-25,400.0,-,XL GROUP LTD,Financials,0.04955
806890,2019-02-21,506.0,-,APARTMENT INVESTMENT DUMMY STOCK D,Real Estate,0.00105
807396,2019-02-22,506.0,-,APARTMENT INVESTMENT DUMMY STOCK D,Real Estate,0.00104
807902,2019-02-25,506.0,-,APARTMENT INVESTMENT DUMMY STOCK D,Real Estate,0.00104
808408,2019-02-26,506.0,-,APARTMENT INVESTMENT DUMMY STOCK D,Real Estate,0.00110
...,...,...,...,...,...,...
1042853,2021-01-08,506.0,-,SL GREEN REALTY CORP CASH ACCRUAL,Real Estate,0.00038
1043359,2021-01-11,506.0,-,SL GREEN REALTY CORP CASH ACCRUAL,Real Estate,0.00038
1043865,2021-01-12,506.0,-,SL GREEN REALTY CORP CASH ACCRUAL,Real Estate,0.00038
1044371,2021-01-13,506.0,-,SL GREEN REALTY CORP CASH ACCRUAL,Real Estate,0.00038


In [247]:
list(df_his_weight[df_his_weight.ticker=="-"].name.unique())

['XL GROUP LTD',
 'APARTMENT INVESTMENT DUMMY STOCK D',
 'DISH NETWORK CORP CLASS A',
 'Aon plc',
 'OCCIDENTAL PETROLEUM CORP',
 'APARTMENT INVESTMENT  CASH ACCRUAL',
 'SL GREEN REALTY CORP CASH ACCRUAL',
 'COOPER COMPANIES INC']

We need to fix this "-" only ticker situation

Fixing XL GROUP LTD

In [250]:
df_his_weight[df_his_weight.name=="XL GROUP LTD"].ticker.unique()

array(['-', 'XL'], dtype=object)

In [251]:
df_his_weight[(df_his_weight.name=="XL GROUP LTD") & (df_his_weight.ticker=="-")]

,date,rank,ticker,name,sector,weight
543629,2016-07-25,400.0,-,XL GROUP LTD,Financials,0.04955


In [252]:
df_his_weight.at[543629,"ticker"]="XL"

In [253]:
df_his_weight[df_his_weight.name=="XL GROUP LTD"].ticker.unique()

array(['XL'], dtype=object)

Fixing APARTMENT INVESTMENT DUMMY STOCK D (Not stable)

In [254]:
df_his_weight[df_his_weight.name=="APARTMENT INVESTMENT DUMMY STOCK D"]
# This is not stable, so we can ignore it. 

,date,rank,ticker,name,sector,weight
806890,2019-02-21,506.0,-,APARTMENT INVESTMENT DUMMY STOCK D,Real Estate,0.00105
807396,2019-02-22,506.0,-,APARTMENT INVESTMENT DUMMY STOCK D,Real Estate,0.00104
807902,2019-02-25,506.0,-,APARTMENT INVESTMENT DUMMY STOCK D,Real Estate,0.00104
808408,2019-02-26,506.0,-,APARTMENT INVESTMENT DUMMY STOCK D,Real Estate,0.00110
808914,2019-02-27,506.0,-,APARTMENT INVESTMENT DUMMY STOCK D,Real Estate,0.00110
809420,2019-02-28,506.0,-,APARTMENT INVESTMENT DUMMY STOCK D,Real Estate,0.00111
809926,2019-03-01,506.0,-,APARTMENT INVESTMENT DUMMY STOCK D,Real Estate,0.00110
810432,2019-03-04,506.0,-,APARTMENT INVESTMENT DUMMY STOCK D,Real Estate,0.00110
810938,2019-03-05,506.0,-,APARTMENT INVESTMENT DUMMY STOCK D,Real Estate,0.00110
811444,2019-03-06,506.0,-,APARTMENT INVESTMENT DUMMY STOCK D,Real Estate,0.00111


Fixing DISH NETWORK CORP CLASS A, we will change "-" to "DISH" for now. 

In [255]:
df_his_weight[df_his_weight.name=="DISH NETWORK CORP CLASS A"].ticker.unique()

array(['DISH', '-', 'DISHR'], dtype=object)

In [256]:
df_his_weight[(df_his_weight.name=="DISH NETWORK CORP CLASS A") & (df_his_weight.ticker=="-")]

,date,rank,ticker,name,sector,weight
899332,2019-11-21,506.0,-,DISH NETWORK CORP CLASS A,Communication,0.00012
899838,2019-11-22,506.0,-,DISH NETWORK CORP CLASS A,Communication,0.00014
900344,2019-11-25,506.0,-,DISH NETWORK CORP CLASS A,Communication,0.00010


In [257]:
df_his_weight.at[899332,"ticker"]="DISH"
df_his_weight.at[899838,"ticker"]="DISH"
df_his_weight.at[900344,"ticker"]="DISH"

In [258]:
df_his_weight[df_his_weight.name=="DISH NETWORK CORP CLASS A"].ticker.unique()

array(['DISH', 'DISHR'], dtype=object)

Fixing Aon plc.

In [259]:
df_his_weight[df_his_weight.name=="Aon plc"].ticker.unique()

array(['-', 'AON'], dtype=object)

In [260]:
df_his_weight[(df_his_weight.name=="Aon plc") & (df_his_weight.ticker=="-")]

,date,rank,ticker,name,sector,weight
943916,2020-04-01,134.0,-,Aon plc,Financials,0.17409


In [261]:
df_his_weight.at[943916,"ticker"]="AON"

In [262]:
df_his_weight[df_his_weight.name=="Aon plc"].ticker.unique()

array(['AON'], dtype=object)

Fixing OCCIDENTAL PETROLEUM CORP

In [263]:
df_his_weight[df_his_weight.name=="OCCIDENTAL PETROLEUM CORP"].ticker.unique()

array(['OXY', '-'], dtype=object)

In [266]:
df_his_weight[(df_his_weight.name=="OCCIDENTAL PETROLEUM CORP") & (df_his_weight.ticker=="-")]

,date,rank,ticker,name,sector,weight
976618,2020-07-02,507.0,-,OCCIDENTAL PETROLEUM CORP,Energy,0.0
977125,2020-07-06,507.0,-,OCCIDENTAL PETROLEUM CORP,Energy,0.0
977632,2020-07-07,507.0,-,OCCIDENTAL PETROLEUM CORP,Energy,0.0
978139,2020-07-08,507.0,-,OCCIDENTAL PETROLEUM CORP,Energy,0.0
978646,2020-07-09,507.0,-,OCCIDENTAL PETROLEUM CORP,Energy,0.0
979153,2020-07-10,507.0,-,OCCIDENTAL PETROLEUM CORP,Energy,0.0
979660,2020-07-13,507.0,-,OCCIDENTAL PETROLEUM CORP,Energy,0.0
980167,2020-07-14,507.0,-,OCCIDENTAL PETROLEUM CORP,Energy,0.0
980674,2020-07-15,507.0,-,OCCIDENTAL PETROLEUM CORP,Energy,0.0
981181,2020-07-16,507.0,-,OCCIDENTAL PETROLEUM CORP,Energy,0.0


In [267]:
df_his_weight.at[976618,"ticker"]="OXY"
df_his_weight.at[977125,"ticker"]="OXY"
df_his_weight.at[977632,"ticker"]="OXY"
df_his_weight.at[978139,"ticker"]="OXY"
df_his_weight.at[978646,"ticker"]="OXY"
df_his_weight.at[979153,"ticker"]="OXY"
df_his_weight.at[979660,"ticker"]="OXY"
df_his_weight.at[980167,"ticker"]="OXY"
df_his_weight.at[980674,"ticker"]="OXY"
df_his_weight.at[981181,"ticker"]="OXY"
df_his_weight.at[981688,"ticker"]="OXY"

In [268]:
df_his_weight[df_his_weight.name=="OCCIDENTAL PETROLEUM CORP"].ticker.unique()

array(['OXY'], dtype=object)

Fixing APARTMENT INVESTMENT  CASH ACCRUAL (Not stable)

In [269]:
df_his_weight[df_his_weight.name=="APARTMENT INVESTMENT  CASH ACCRUAL"].ticker.unique()

array(['-'], dtype=object)

In [271]:
df_his_weight[(df_his_weight.name=="APARTMENT INVESTMENT  CASH ACCRUAL") & (df_his_weight.ticker=="-")]
# Not a stable stock, ignoring.

,date,rank,ticker,name,sector,weight
1021608,2020-11-06,506.0,-,APARTMENT INVESTMENT CASH ACCRUAL,Real Estate,0.00365
1022114,2020-11-09,506.0,-,APARTMENT INVESTMENT CASH ACCRUAL,Real Estate,0.00360
1022620,2020-11-10,506.0,-,APARTMENT INVESTMENT CASH ACCRUAL,Real Estate,0.00361
1023126,2020-11-11,506.0,-,APARTMENT INVESTMENT CASH ACCRUAL,Real Estate,0.00358
1023632,2020-11-12,506.0,-,APARTMENT INVESTMENT CASH ACCRUAL,Real Estate,0.00361
1024138,2020-11-13,506.0,-,APARTMENT INVESTMENT CASH ACCRUAL,Real Estate,0.00356
1024644,2020-11-16,506.0,-,APARTMENT INVESTMENT CASH ACCRUAL,Real Estate,0.00352
1025150,2020-11-17,506.0,-,APARTMENT INVESTMENT CASH ACCRUAL,Real Estate,0.00353
1025656,2020-11-18,506.0,-,APARTMENT INVESTMENT CASH ACCRUAL,Real Estate,0.00357
1026162,2020-11-19,506.0,-,APARTMENT INVESTMENT CASH ACCRUAL,Real Estate,0.00355


Fixing SL GREEN REALTY CORP CASH ACCRUAL (Not Stable)

In [ ]:
df_his_weight[df_his_weight.name=="SL GREEN REALTY CORP CASH ACCRUAL"].ticker.unique()

array(['-'], dtype=object)

In [273]:
df_his_weight[(df_his_weight.name=="SL GREEN REALTY CORP CASH ACCRUAL") & (df_his_weight.ticker=="-")]
# Not a stable stock, ignoring.

,date,rank,ticker,name,sector,weight
1034247,2020-12-14,506.0,-,SL GREEN REALTY CORP CASH ACCRUAL,Real Estate,0.00039
1034754,2020-12-15,507.0,-,SL GREEN REALTY CORP CASH ACCRUAL,Real Estate,0.00038
1035261,2020-12-16,507.0,-,SL GREEN REALTY CORP CASH ACCRUAL,Real Estate,0.00039
1035768,2020-12-17,507.0,-,SL GREEN REALTY CORP CASH ACCRUAL,Real Estate,0.00039
1036274,2020-12-18,506.0,-,SL GREEN REALTY CORP CASH ACCRUAL,Real Estate,0.00039
1036780,2020-12-21,506.0,-,SL GREEN REALTY CORP CASH ACCRUAL,Real Estate,0.00039
1037286,2020-12-22,506.0,-,SL GREEN REALTY CORP CASH ACCRUAL,Real Estate,0.00040
1037792,2020-12-23,506.0,-,SL GREEN REALTY CORP CASH ACCRUAL,Real Estate,0.00040
1038298,2020-12-24,506.0,-,SL GREEN REALTY CORP CASH ACCRUAL,Real Estate,0.00040
1038804,2020-12-28,506.0,-,SL GREEN REALTY CORP CASH ACCRUAL,Real Estate,0.00039


Fixing COOPER COMPANIES INC (Not stable)

In [274]:
df_his_weight[df_his_weight.name=="COOPER COMPANIES INC"].ticker.unique()

array(['-'], dtype=object)

In [275]:
df_his_weight[(df_his_weight.name=="COOPER COMPANIES INC") & (df_his_weight.ticker=="-")]
# Not a stable stock, ignoring.

,date,rank,ticker,name,sector,weight
1438288,2024-02-16,355.0,-,COOPER COMPANIES INC,Health Care,0.04362


Back up progress

In [277]:
df_his_weight.to_csv("../../processed/his_weight_fix_tic_in_process.csv", index=False)

In above, we now want to drop APARTMENT INVESTMENT DUMMY STOCK D, APARTMENT INVESTMENT  CASH ACCRUAL, SL GREEN REALTY CORP CASH ACCRUAL, and COOPER COMPANIES INC. We should be able to do this by simply remover "-" ticker rows. 

In [280]:
list(df_his_weight[df_his_weight.ticker=="-"].name.unique())

['APARTMENT INVESTMENT DUMMY STOCK D',
 'APARTMENT INVESTMENT  CASH ACCRUAL',
 'SL GREEN REALTY CORP CASH ACCRUAL',
 'COOPER COMPANIES INC']

In [281]:
df_his_weight=df_his_weight[df_his_weight.ticker!="-"]

In [282]:
list(df_his_weight[df_his_weight.ticker=="-"].name.unique())

[]

Back up pregress 

In [283]:
df_his_weight.to_csv("../../processed/his_weight_fix_tic_in_process.csv", index=False)

In [284]:
lst_weight_ticker=list(df_his_weight.ticker.unique())
lst_weight_ticker

['XOM',
 'PG',
 'JNJ',
 'T',
 'CVX',
 'MSFT',
 'GEC',
 'IBM',
 'WMT',
 'PFE',
 'JPM',
 'CSCO',
 'ABT',
 'HPQ',
 'KO',
 'VZ',
 'GOOGL',
 'AAPL*',
 'WFC*',
 'PEP',
 'PM',
 'INCO',
 'COP',
 'ORCL',
 'MCD',
 'MRK',
 'AMGN',
 'WYE',
 'QCI',
 'SLBA',
 'GILD',
 'UTX',
 'OXY',
 'BMY',
 'UPS',
 'CMCSA',
 'BAC',
 'MON',
 'KFT',
 'CVS',
 'DIS',
 'MDT',
 'MMM',
 'LLY',
 'HD',
 'BAX',
 'GS',
 'EXC',
 'UNH',
 'MO',
 'TWX',
 'CL',
 'BA',
 'BK',
 'LOM',
 'DVN',
 'WAG',
 'LOW',
 'USB',
 'SO',
 'APA',
 'EMR',
 'CELG',
 'ALD',
 'TGT',
 'MWZ',
 'PA9',
 'EMC',
 'MHS',
 'UNP',
 'GD',
 'MS',
 'XTO',
 'KMB',
 'ANTM',
 'NEE',
 'RTN1',
 'DD',
 'D',
 'GIS',
 'COST',
 'AEC1',
 'TXN',
 '0R01',
 'COV',
 'NEM',
 'MRO',
 'PX',
 'DUK',
 'AMZN',
 'GENZ',
 'CAT',
 'BNI',
 'ADP',
 'NKE',
 'BDX',
 'ADM*',
 'EOG',
 'AAZ',
 'DELL',
 'AABA',
 'PEG',
 'FDX',
 'HES',
 'NTH',
 'GLW',
 'HAL',
 'WM',
 'FE',
 'CB',
 'TMO',
 'DE',
 'NCRA',
 'KR',
 'ETR',
 'DHR',
 'AET',
 'NSC',
 'BIIB',
 'PCG',
 'PNC',
 'CAH',
 'SYY',
 'BSX',
 'ESR

In [285]:
reg_allow=re.compile("^[A-Za-z0-9]*$")
lst_weight_ticker_weird=[tic for tic in lst_weight_ticker if not bool(reg_allow.fullmatch(tic))]

In [286]:
lst_weight_ticker_weird

['AAPL*',
 'WFC*',
 'ADM*',
 'PRU*',
 'FCX*',
 'AIG*',
 'MIL.',
 'NSM.2',
 'GAS.2',
 'TWX*',
 'MOS*',
 'KMI WS',
 'S*',
 'FB*',
 'GOOG*',
 'BF/B',
 'BF.B',
 'TLN WI',
 'GCI.WI',
 'BLD WI',
 'CC WI',
 'CCP W',
 'SYF-W',
 'UAC/C',
 'HWM WI',
 'ARNC WI',
 'RTX US',
 'OXY WS WI',
 'OXY WS',
 'VTRS US',
 'MBC WI']

Good, "-" only ticker is no longer present. 

We turn our attention to tickers from df_com_CIK_SIC_NAICS

In [348]:
lst_tech_adj_ticker = list(df_com_CIK_SIC_NAICS_rele.tickers.unique())
lst_tech_adj_ticker

['AAPL',
 'MSFT',
 'NVDA',
 'GOOGL,GOOG',
 'META',
 'TSM,TSMWF',
 'AVGO',
 'ORCL',
 'ASML,ASMLF',
 'CRM',
 'SAP,SAPGF',
 'ADBE',
 'AMD',
 'TMO',
 'TMUS',
 'CSCO',
 'DHR',
 'INTU',
 'TXN',
 'IBM',
 'AMAT',
 'NOW',
 'VZ',
 'CAT',
 'CMCSA,CCZ',
 'T,TBB,TBC,T-PA,T-PC',
 'ARM',
 'MU',
 'ETN',
 'LRCX',
 'ANET',
 'ADI',
 'PANW',
 'MDT',
 'ADP',
 'DE',
 'SHOP',
 'INFY',
 'INTC',
 'SNPS',
 'APH',
 'DELL',
 'CDNS',
 'TT',
 'PLTR',
 'ITW',
 'SPOT',
 'NXPI',
 'CRWD',
 'WDAY',
 'CARR',
 'MRVL',
 'ROP',
 'NTES,NETTF',
 'FTNT',
 'ADSK',
 'AMX,AMXOF',
 'CHTR',
 'TTD',
 'JCI',
 'MPWR',
 'MCHP',
 'SNOW',
 'CMI',
 'SQ,BSQKZ',
 'TEAM',
 'EA',
 'DDOG',
 'VRSK',
 'AME',
 'CTSH',
 'FLUT',
 'IR',
 'SMCI',
 'HPQ',
 'BKR',
 'ATEYY,ADTTF',
 'ON',
 'XYL',
 'WIT',
 'BCE,BECEF,BCAEF,BCEFF,BCEIF,BCEPF,BCEXF,BCPPF',
 'VEEV',
 'ORAN,FNCTF',
 'CHT',
 'BIDU,BAIDF',
 'VRT',
 'ROK',
 'ZS',
 'APP',
 'ANSS',
 'NET',
 'STM,STMEF',
 'NTAP',
 'VLTO',
 'TTWO',
 'RBLX',
 'MSTR',
 'TEF,TEFOF',
 'VOD,VODPF',
 'HUBS',
 'GFS',
 'TYL

In above is a list of strings (there is one float nan) each representing some tickers (might not be all) of a company considered as relevant to us (tech industry adjacent). Let's make it a list where each string is one singular ticker instead. 

Belows is retrieved manually: 

445 WalkMe Ltd.: WKME

872, 1518 MariaDB plc: MRDB

949 Nano Labs Ltd: NA 

1165 2U, LLC: TWOUQ 

1213 Sixty Six Oilfield Services, Inc.: SSOF 

1221 EBIX INC: EBIXQ

1227 Gamehaus Holdings Inc.: GMHS

1228 NetClass Technology Inc: NTCL

1229 Harden Technologies Inc.: HAHA

1231 WeRide Inc.: WRD

1236 UEOPLE Technology Holding Ltd: UEOP

1242 Scripps Safe, Inc.: SSAI

1308 Fang Holdings Ltd: SFUNY

1319 WEARABLE HEALTH SOLUTIONS, INC.: WHSI

1385 DecisionPoint Systems, Inc.: DPSI 

1505 Linktory Inc.: LKRY

Due to fact that it is hard to make sure they do not have otherwise tickers already recorded (situations like repeated company name entry), and the list is not too long, I updated the dataframe manually. 
Update csv. 

In [162]:
df_com_CIK_SIC_NAICS_rele.to_csv("../../processed/com_CIK_SIC_NAICS_rele.csv",index=False)
# df_com_CIK_SIC_NAICS_rele= pd.read_csv("../../processed/com_CIK_SIC_NAICS_rele.csv")

In [349]:
lst_tech_adj_ticker = list(df_com_CIK_SIC_NAICS_rele.tickers.unique())
lst_tech_adj_ticker

['AAPL',
 'MSFT',
 'NVDA',
 'GOOGL,GOOG',
 'META',
 'TSM,TSMWF',
 'AVGO',
 'ORCL',
 'ASML,ASMLF',
 'CRM',
 'SAP,SAPGF',
 'ADBE',
 'AMD',
 'TMO',
 'TMUS',
 'CSCO',
 'DHR',
 'INTU',
 'TXN',
 'IBM',
 'AMAT',
 'NOW',
 'VZ',
 'CAT',
 'CMCSA,CCZ',
 'T,TBB,TBC,T-PA,T-PC',
 'ARM',
 'MU',
 'ETN',
 'LRCX',
 'ANET',
 'ADI',
 'PANW',
 'MDT',
 'ADP',
 'DE',
 'SHOP',
 'INFY',
 'INTC',
 'SNPS',
 'APH',
 'DELL',
 'CDNS',
 'TT',
 'PLTR',
 'ITW',
 'SPOT',
 'NXPI',
 'CRWD',
 'WDAY',
 'CARR',
 'MRVL',
 'ROP',
 'NTES,NETTF',
 'FTNT',
 'ADSK',
 'AMX,AMXOF',
 'CHTR',
 'TTD',
 'JCI',
 'MPWR',
 'MCHP',
 'SNOW',
 'CMI',
 'SQ,BSQKZ',
 'TEAM',
 'EA',
 'DDOG',
 'VRSK',
 'AME',
 'CTSH',
 'FLUT',
 'IR',
 'SMCI',
 'HPQ',
 'BKR',
 'ATEYY,ADTTF',
 'ON',
 'XYL',
 'WIT',
 'BCE,BECEF,BCAEF,BCEFF,BCEIF,BCEPF,BCEXF,BCPPF',
 'VEEV',
 'ORAN,FNCTF',
 'CHT',
 'BIDU,BAIDF',
 'VRT',
 'ROK',
 'ZS',
 'APP',
 'ANSS',
 'NET',
 'STM,STMEF',
 'NTAP',
 'VLTO',
 'TTWO',
 'RBLX',
 'MSTR',
 'TEF,TEFOF',
 'VOD,VODPF',
 'HUBS',
 'GFS',
 'TYL

In [350]:
for i in range(len(lst_tech_adj_ticker)): 
    if type(lst_tech_adj_ticker[i]) != str: 
        print(i, lst_tech_adj_ticker[i], type(lst_tech_adj_ticker[i]))
        
print("no ono")

no ono


In [165]:
lst_tech_adj_ticker_SINGULAR = []

for i in range(len(lst_tech_adj_ticker)): 
    lst_tech_adj_ticker_SINGULAR = lst_tech_adj_ticker_SINGULAR + lst_tech_adj_ticker[i].split(",")
    
lst_tech_adj_ticker_SINGULAR

['AAPL',
 'MSFT',
 'NVDA',
 'GOOGL',
 'GOOG',
 'META',
 'TSM',
 'TSMWF',
 'AVGO',
 'ORCL',
 'ASML',
 'ASMLF',
 'CRM',
 'SAP',
 'SAPGF',
 'ADBE',
 'AMD',
 'TMO',
 'TMUS',
 'CSCO',
 'DHR',
 'INTU',
 'TXN',
 'IBM',
 'AMAT',
 'NOW',
 'VZ',
 'CAT',
 'CMCSA',
 'CCZ',
 'T',
 'TBB',
 'TBC',
 'T-PA',
 'T-PC',
 'ARM',
 'MU',
 'ETN',
 'LRCX',
 'ANET',
 'ADI',
 'PANW',
 'MDT',
 'ADP',
 'DE',
 'SHOP',
 'INFY',
 'INTC',
 'SNPS',
 'APH',
 'DELL',
 'CDNS',
 'TT',
 'PLTR',
 'ITW',
 'SPOT',
 'NXPI',
 'CRWD',
 'WDAY',
 'CARR',
 'MRVL',
 'ROP',
 'NTES',
 'NETTF',
 'FTNT',
 'ADSK',
 'AMX',
 'AMXOF',
 'CHTR',
 'TTD',
 'JCI',
 'MPWR',
 'MCHP',
 'SNOW',
 'CMI',
 'SQ',
 'BSQKZ',
 'TEAM',
 'EA',
 'DDOG',
 'VRSK',
 'AME',
 'CTSH',
 'FLUT',
 'IR',
 'SMCI',
 'HPQ',
 'BKR',
 'ATEYY',
 'ADTTF',
 'ON',
 'XYL',
 'WIT',
 'BCE',
 'BECEF',
 'BCAEF',
 'BCEFF',
 'BCEIF',
 'BCEPF',
 'BCEXF',
 'BCPPF',
 'VEEV',
 'ORAN',
 'FNCTF',
 'CHT',
 'BIDU',
 'BAIDF',
 'VRT',
 'ROK',
 'ZS',
 'APP',
 'ANSS',
 'NET',
 'STM',
 'STMEF',
 'N

In [166]:
len(set(lst_tech_adj_ticker_SINGULAR))==len(lst_tech_adj_ticker_SINGULAR)

True

Let's check for weird tickers as well. 

In [209]:
lst_tech_adj_ticker_SINGULAR_weird=[tic for tic in lst_tech_adj_ticker_SINGULAR if not bool(reg_allow.fullmatch(tic))]

In [210]:
lst_tech_adj_ticker_SINGULAR_weird

['T-PA',
 'T-PC',
 'HPE-PC',
 'MOG-A',
 'MOG-B',
 'BC-PA',
 'BC-PB',
 'BC-PC',
 'TDS-PU',
 'TDS-PV',
 'GRND-WT',
 'IONQ-WT',
 'ETWO-WT',
 'PBI-PB',
 'GB-WT',
 'NPWR-WT',
 'GTN-A',
 'OUST-WT',
 'OUST-WTA',
 'SES-WT',
 'BBAI-WT',
 'CTV-WT',
 'VLN-WT',
 'QBTS-WT',
 'PERF-WT',
 'FREY-WT',
 'GCTS-WT',
 'AMPX-WT',
 'SST-WT',
 'GWH-WT',
 'AP-WT',
 'LGL-WT',
 'GPUS-PD']

In [319]:
set_tech_adj_ticker_wrd_symb=set()

for tic in lst_tech_adj_ticker_SINGULAR_weird: 
    # print(tic)
    wrd_symb=re.findall("[A-Za-z0-9]*([^A-Za-z0-9])",tic)[0]
    set_tech_adj_ticker_wrd_symb.add(wrd_symb)
    
set_tech_adj_ticker_wrd_symb

{'-'}

It seems this all weird tickers here are standard in the way that they are the ticker symbol followed by a special code separated by "-". Let's see if the ticker symbols prior to the special codes are included already. 

In [302]:
for tic in lst_tech_adj_ticker_SINGULAR_weird: 
    com_tic=re.findall("([A-Za-z0-9]*)\-",tic)[0]
    if com_tic not in lst_tech_adj_ticker_SINGULAR: 
        print("ono:",tic,com_tic)
print("no ono")

ono: MOG-A MOG
ono: MOG-B MOG
no ono


<>:2: SyntaxWarning: invalid escape sequence '\-'
<>:2: SyntaxWarning: invalid escape sequence '\-'
/tmp/ipykernel_13657/1261352098.py:2: SyntaxWarning: invalid escape sequence '\-'
  com_tic=re.findall("([A-Za-z0-9]*)\-",tic)[0]


Let's add the missing ticker into the list

In [303]:
for tic in lst_tech_adj_ticker_SINGULAR_weird: 
    com_tic=re.findall("([A-Za-z0-9]*)\-",tic)[0]
    if com_tic not in lst_tech_adj_ticker_SINGULAR: 
        lst_tech_adj_ticker_SINGULAR.append(com_tic)

<>:2: SyntaxWarning: invalid escape sequence '\-'
<>:2: SyntaxWarning: invalid escape sequence '\-'
/tmp/ipykernel_13657/1734321752.py:2: SyntaxWarning: invalid escape sequence '\-'
  com_tic=re.findall("([A-Za-z0-9]*)\-",tic)[0]


check again

In [304]:
for tic in lst_tech_adj_ticker_SINGULAR_weird: 
    com_tic=re.findall("([A-Za-z0-9]*)\-",tic)[0]
    if com_tic not in lst_tech_adj_ticker_SINGULAR: 
        print("ono:",tic,com_tic)
print("no ono")

no ono


<>:2: SyntaxWarning: invalid escape sequence '\-'
<>:2: SyntaxWarning: invalid escape sequence '\-'
/tmp/ipykernel_13657/1261352098.py:2: SyntaxWarning: invalid escape sequence '\-'
  com_tic=re.findall("([A-Za-z0-9]*)\-",tic)[0]


We turn our attention back to lst_weight_ticker_weird, we will add the company tickers into the list as well. 

In [307]:
re.findall("([A-Za-z0-9]*)[^A-Za-z0-9]", "AAPL*")

['AAPL']

In [308]:
for tic in lst_weight_ticker_weird: 
    # print(tic)
    com_tic=re.findall("([A-Za-z0-9]*)[^A-Za-z0-9]",tic)[0]
    if com_tic not in lst_weight_ticker: 
        print("ono:",tic,com_tic)
print("no ono")

ono: MIL. MIL
ono: NSM.2 NSM
ono: BF/B BF
ono: BF.B BF
ono: TLN WI TLN
ono: BLD WI BLD
ono: CC WI CC
ono: CCP W CCP
ono: MBC WI MBC
no ono


In [309]:
for tic in lst_weight_ticker_weird: 
    # print(tic)
    com_tic=re.findall("([A-Za-z0-9]*)[^A-Za-z0-9]",tic)[0]
    if com_tic not in lst_weight_ticker: 
        lst_weight_ticker.append(com_tic)


In [310]:
for tic in lst_weight_ticker_weird: 
    # print(tic)
    com_tic=re.findall("([A-Za-z0-9]*)[^A-Za-z0-9]",tic)[0]
    if com_tic not in lst_weight_ticker: 
        print("ono:",tic,com_tic)
print("no ono")

no ono


In [311]:
lst_weight_ticker_weird

['AAPL*',
 'WFC*',
 'ADM*',
 'PRU*',
 'FCX*',
 'AIG*',
 'MIL.',
 'NSM.2',
 'GAS.2',
 'TWX*',
 'MOS*',
 'KMI WS',
 'S*',
 'FB*',
 'GOOG*',
 'BF/B',
 'BF.B',
 'TLN WI',
 'GCI.WI',
 'BLD WI',
 'CC WI',
 'CCP W',
 'SYF-W',
 'UAC/C',
 'HWM WI',
 'ARNC WI',
 'RTX US',
 'OXY WS WI',
 'OXY WS',
 'VTRS US',
 'MBC WI']

In [315]:
set_weight_ticker_wrd_symb=set()

for tic in lst_weight_ticker_weird: 
    # print(tic)
    wrd_symb=re.findall("[A-Za-z0-9]*([^A-Za-z0-9])",tic)[0]
    set_weight_ticker_wrd_symb.add(wrd_symb)
    
set_weight_ticker_wrd_symb

{' ', '*', '-', '.', '/'}

In [320]:
lst_weight_ticker_changed=copy.deepcopy(lst_weight_ticker)

In [ ]:
for tic in lst_weight_ticker_changed: 
    for symb in set_weight_ticker_wrd_symb: 
        tic.replace(symb,"-")

In [322]:
set_tech_sp_ticker=set(lst_tech_adj_ticker_SINGULAR) & set(lst_weight_ticker_changed)

Below is the set of all S an P 500 tech adjacent company tickers 

In [323]:
set_tech_sp_ticker

{'AAPL',
 'ADBE',
 'ADI',
 'ADP',
 'ADSK',
 'ALTR',
 'AMAT',
 'AMD',
 'AME',
 'ANET',
 'ANSS',
 'APH',
 'AVGO',
 'AYI',
 'BKR',
 'CARR',
 'CAT',
 'CDNS',
 'CHTR',
 'CIEN',
 'CMCSA',
 'CMI',
 'CRM',
 'CRWD',
 'CSCO',
 'CTSH',
 'DAY',
 'DE',
 'DELL',
 'DHR',
 'DNOW',
 'DOV',
 'DV',
 'DXC',
 'EA',
 'ENPH',
 'EPAM',
 'ETN',
 'FDS',
 'FFIV',
 'FLS',
 'FOX',
 'FOXA',
 'FSLR',
 'FTI',
 'FTNT',
 'FTV',
 'GDDY',
 'GEN',
 'GOOG',
 'GOOGL',
 'HAS',
 'HCP',
 'HPE',
 'HPQ',
 'HUBB',
 'IBM',
 'IEX',
 'INTC',
 'INTU',
 'IPGP',
 'IR',
 'ITT',
 'ITW',
 'JCI',
 'JKHY',
 'JNPR',
 'KEYS',
 'LDOS',
 'LRCX',
 'LUMN',
 'MCHP',
 'MDT',
 'META',
 'MPWR',
 'MSFT',
 'MTCH',
 'MTW',
 'MU',
 'NDSN',
 'NOV',
 'NOW',
 'NTAP',
 'NVDA',
 'NXPI',
 'ON',
 'ORCL',
 'PANW',
 'PARA',
 'PAYC',
 'PBI',
 'PLTR',
 'PNR',
 'PTC',
 'QRVO',
 'ROK',
 'ROP',
 'S',
 'SEDG',
 'SLE',
 'SMCI',
 'SNPS',
 'STI',
 'STX',
 'SWKS',
 'T',
 'TDC',
 'TER',
 'TGNA',
 'TMO',
 'TMUS',
 'TRIP',
 'TRMB',
 'TT',
 'TTWO',
 'TXN',
 'TYL',
 'VLTO',
 'V

In [339]:
set_tech_sp_wrd_symb=set()

for tic in set_tech_sp_ticker: 
    # print(tic)
    if re.findall("[A-Za-z0-9]*([^A-Za-z0-9])",tic):
        wrd_symb=re.findall("[A-Za-z0-9]*([^A-Za-z0-9])",tic)[0]
        set_tech_sp_wrd_symb.add(wrd_symb)
    
set_tech_sp_wrd_symb

set()

In [366]:
len(set_tech_sp_ticker)

126

In [345]:
lst_tech_adj_ticker

['AAPL',
 'MSFT',
 'NVDA',
 'GOOGL,GOOG',
 'META',
 'TSM,TSMWF',
 'AVGO',
 'ORCL',
 'ASML,ASMLF',
 'CRM',
 'SAP,SAPGF',
 'ADBE',
 'AMD',
 'TMO',
 'TMUS',
 'CSCO',
 'DHR',
 'INTU',
 'TXN',
 'IBM',
 'AMAT',
 'NOW',
 'VZ',
 'CAT',
 'CMCSA,CCZ',
 'T,TBB,TBC,T-PA,T-PC',
 'ARM',
 'MU',
 'ETN',
 'LRCX',
 'ANET',
 'ADI',
 'PANW',
 'MDT',
 'ADP',
 'DE',
 'SHOP',
 'INFY',
 'INTC',
 'SNPS',
 'APH',
 'DELL',
 'CDNS',
 'TT',
 'PLTR',
 'ITW',
 'SPOT',
 'NXPI',
 'CRWD',
 'WDAY',
 'CARR',
 'MRVL',
 'ROP',
 'NTES,NETTF',
 'FTNT',
 'ADSK',
 'AMX,AMXOF',
 'CHTR',
 'TTD',
 'JCI',
 'MPWR',
 'MCHP',
 'SNOW',
 'CMI',
 'SQ,BSQKZ',
 'TEAM',
 'EA',
 'DDOG',
 'VRSK',
 'AME',
 'CTSH',
 'FLUT',
 'IR',
 'SMCI',
 'HPQ',
 'BKR',
 'ATEYY,ADTTF',
 'ON',
 'XYL',
 'WIT',
 'BCE,BECEF,BCAEF,BCEFF,BCEIF,BCEPF,BCEXF,BCPPF',
 'VEEV',
 'ORAN,FNCTF',
 'CHT',
 'BIDU,BAIDF',
 'VRT',
 'ROK',
 'ZS',
 'APP',
 'ANSS',
 'NET',
 'STM,STMEF',
 'NTAP',
 'VLTO',
 'TTWO',
 'RBLX',
 'MSTR',
 'TEF,TEFOF',
 'VOD,VODPF',
 'HUBS',
 'GFS',
 'TYL

In [357]:
re.findall("([A-Za-z0-9]*)[^A-Za-z0-9]",'GOOGL,GOOG')[0] in set_tech_sp_ticker

True

In [358]:
re.findall("([A-Za-z0-9]*)[^A-Za-z0-9]","AAPL")

[]

In [360]:
lst_tickers_keep=[]

for tickers in lst_tech_adj_ticker: 
    # print(tickers)
    lst_tick_proc=re.findall("([A-Za-z0-9]*)[^A-Za-z0-9]",tickers)
    if len(lst_tick_proc) == 0: 
        if tickers in set_tech_sp_ticker:
            lst_tickers_keep.append(tickers)
    else: 
        if lst_tick_proc[0] in set_tech_sp_ticker: 
            lst_tickers_keep.append(tickers)

In [370]:
lst_tickers_keep=[]

for tickers in lst_tech_adj_ticker: 
    for tick in tickers.split(","): 
        lst_tick_proc=re.findall("([A-Za-z0-9]*)[^A-Za-z0-9]",tick)
        if len(lst_tick_proc) == 0: 
            if tick in set_tech_sp_ticker:
                lst_tickers_keep.append(tickers)
        else: 
            if lst_tick_proc[0] in set_tech_sp_ticker: 
                lst_tickers_keep.append(tickers)
                
lst_tickers_keep

['AAPL',
 'MSFT',
 'NVDA',
 'GOOGL,GOOG',
 'GOOGL,GOOG',
 'META',
 'AVGO',
 'ORCL',
 'CRM',
 'ADBE',
 'AMD',
 'TMO',
 'TMUS',
 'CSCO',
 'DHR',
 'INTU',
 'TXN',
 'IBM',
 'AMAT',
 'NOW',
 'VZ',
 'CAT',
 'CMCSA,CCZ',
 'T,TBB,TBC,T-PA,T-PC',
 'T,TBB,TBC,T-PA,T-PC',
 'T,TBB,TBC,T-PA,T-PC',
 'MU',
 'ETN',
 'LRCX',
 'ANET',
 'ADI',
 'PANW',
 'MDT',
 'ADP',
 'DE',
 'INTC',
 'SNPS',
 'APH',
 'DELL',
 'CDNS',
 'TT',
 'PLTR',
 'ITW',
 'NXPI',
 'CRWD',
 'CARR',
 'ROP',
 'FTNT',
 'ADSK',
 'CHTR',
 'JCI',
 'MPWR',
 'MCHP',
 'CMI',
 'EA',
 'VRSK',
 'AME',
 'CTSH',
 'IR',
 'SMCI',
 'HPQ',
 'BKR',
 'ON',
 'XYL',
 'ROK',
 'ANSS',
 'NTAP',
 'VLTO',
 'TTWO',
 'TYL',
 'DOV',
 'HPE,HPE-PC',
 'HPE,HPE-PC',
 'FTV',
 'FSLR',
 'KEYS',
 'GDDY',
 'TER',
 'STX',
 'WDC',
 'PTC',
 'HUBB',
 'LDOS',
 'WBD',
 'ZBRA',
 'FOXA,FOX',
 'FOXA,FOX',
 'VRSN',
 'SWKS',
 'ENPH',
 'FDS',
 'GEN',
 'IEX',
 'PNR',
 'NDSN',
 'TRMB',
 'JNPR',
 'JKHY',
 'FTI',
 'EPAM',
 'FFIV',
 'ITT',
 'QRVO',
 'HAS',
 'PAYC',
 'MTCH',
 'DAY',
 'PARA,

In [369]:
set(lst_tickers_keep)

{'AAPL',
 'ADBE',
 'ADI',
 'ADP',
 'ADSK',
 'ALTR',
 'AMAT',
 'AMD',
 'AME',
 'ANET',
 'ANSS',
 'APH',
 'AVGO',
 'AYI',
 'BKR',
 'CARR',
 'CAT',
 'CDNS',
 'CHTR',
 'CIEN',
 'CMCSA,CCZ',
 'CMI',
 'CRM',
 'CRWD',
 'CSCO',
 'CTSH',
 'DAY',
 'DE',
 'DELL',
 'DHR',
 'DNOW',
 'DOV',
 'DV',
 'DXC',
 'EA',
 'ENPH',
 'EPAM',
 'ETN',
 'FDS',
 'FFIV',
 'FLS',
 'FOXA,FOX',
 'FSLR',
 'FTI',
 'FTNT',
 'FTV',
 'GDDY',
 'GEN',
 'GOOGL,GOOG',
 'HAS',
 'HCP',
 'HPE,HPE-PC',
 'HPQ',
 'HUBB',
 'IBM',
 'IEX',
 'INTC',
 'INTU',
 'IPGP',
 'IR',
 'ITT',
 'ITW',
 'JCI',
 'JKHY',
 'JNPR',
 'KEYS',
 'LDOS',
 'LRCX',
 'LUMN',
 'MCHP',
 'MDT',
 'META',
 'MPWR',
 'MSFT',
 'MTCH',
 'MTW',
 'MU',
 'NDSN',
 'NOV',
 'NOW',
 'NTAP',
 'NVDA',
 'NXPI',
 'ON',
 'ORCL',
 'PANW',
 'PARA,PARAA',
 'PAYC',
 'PBI,PBI-PB',
 'PLTR',
 'PNR',
 'PTC',
 'QRVO',
 'ROK',
 'ROP',
 'S',
 'SEDG',
 'SLE',
 'SMCI',
 'SNPS',
 'STI',
 'STX',
 'SWKS',
 'T,TBB,TBC,T-PA,T-PC',
 'TDC',
 'TER',
 'TGNA',
 'TMO',
 'TMUS',
 'TRIP',
 'TRMB',
 'TT',
 'T

In [375]:
df_tech_sp_com_CIK_SIC_NAICS=df_com_CIK_SIC_NAICS_rele[df_com_CIK_SIC_NAICS_rele.tickers.isin(lst_tickers_keep)].drop_duplicates()
df_tech_sp_com_CIK_SIC_NAICS

,cik,name,entityType,sic,sicDescription,ownerOrg,insiderTransactionForOwnerExists,insiderTransactionForIssuerExists,tickers,exchanges,...,mailing_stateOrCountry,mailing_zipCode,mailing_stateOrCountryDescription,business_street1,business_street2,business_city,business_stateOrCountry,business_zipCode,business_stateOrCountryDescription,naics
0,320193,Apple Inc.,operating,3571.0,Electronic Computers,06 Technology,0,1,AAPL,Nasdaq,...,CA,95014,CA,ONE APPLE PARK WAY,NaN,CUPERTINO,CA,95014,CA,334111
1,789019,MICROSOFT CORP,operating,7372.0,Services-Prepackaged Software,06 Technology,1,1,MSFT,Nasdaq,...,WA,98052-6399,WA,ONE MICROSOFT WAY,NaN,REDMOND,WA,98052-6399,WA,"511210,334614"
2,1045810,NVIDIA CORP,operating,3674.0,Semiconductors & Related Devices,04 Manufacturing,1,1,NVDA,Nasdaq,...,CA,95051,CA,2788 SAN TOMAS EXPRESSWAY,NaN,SANTA CLARA,CA,95051,CA,334413
3,1652044,Alphabet Inc.,operating,7370.0,"Services-Computer Programming, Data Processing...",06 Technology,1,1,"GOOGL,GOOG","Nasdaq,Nasdaq",...,CA,94043,CA,1600 AMPHITHEATRE PARKWAY,NaN,MOUNTAIN VIEW,CA,94043,CA,"518210,541511"
4,1326801,"Meta Platforms, Inc.",operating,7370.0,"Services-Computer Programming, Data Processing...",06 Technology,0,1,META,Nasdaq,...,CA,94025,CA,1 META WAY,NaN,MENLO PARK,CA,94025,CA,"518210,541511"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,78814,PITNEY BOWES INC /DE/,operating,3579.0,"Office Machines, NEC",06 Technology,0,1,"PBI,PBI-PB","NYSE,NYSE",...,CT,06926-0700,CT,PITNEY BOWES INC,3001 SUMMER STREET,STAMFORD,CT,06926-0700,CT,"339940,333318,334519"
451,1770450,Xerox Holdings Corp,operating,3577.0,"Computer Peripheral Equipment, NEC",06 Technology,0,1,XRX,Nasdaq,...,CT,06851,CT,201 MERRITT 7,NaN,NORWALK,CT,06851,CT,"334418,333316,334613,334118"
606,61986,MANITOWOC CO INC,operating,3531.0,Construction Machinery & Equip,06 Technology,1,1,MTW,NYSE,...,WI,53224,WI,11270 WEST PARK PLACE,SUITE 10000,MILWAUKEE,WI,53224,WI,"333120,333923,336510"
910,1881551,Solidion Technology Inc.,operating,3690.0,"Miscellaneous Electrical Machinery, Equipment ...",04 Manufacturing,0,1,STI,Nasdaq,...,TX,75240,TX,13355 NOEL ROAD,SUITE 1100,DALLAS,TX,75240,TX,"333618,333318,333992,335129,335999"


In [376]:
df_tech_sp_com_CIK_SIC_NAICS.to_csv("../../processed/tech_sp500_com_CIK_SIC_NAICS.csv",index=False)